# RF based on 2000 selected SNPs from both distribution tails of rel. fitness betas


## Import packages

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(color_codes=True)
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(13,10)})
pd.set_option('display.max_columns', 999)

## Select specific SNPs
Select 1000 SNPs with highest & lowest selection coefficient (from both distribution tails):

In [ ]:
betas = pd.read_csv('/Carnegie/DPB/Data/Shared/Labs/Moi/Everyone/deepselection/randomForest/betas_woNAs_55climvars_rFit.txt', sep='\t')
betas.rename(columns={'clim-bio18.assoc_y':'clim-bio18'}, inplace=True)
betas.drop(['clim-bio18.assoc_x'],axis=1, inplace=True)

In [ ]:
# Extract mlp and mli datasets
MLP = betas[['rs', 'rFitness2_mlp']]
MLI = betas[['rs', 'rFitness2_mli']]
THP = betas[['rs', 'rFitness2_thp']]
THI = betas[['rs', 'rFitness2_thi']]
THI

In [ ]:
# check distribution of betas from GWAS to rel. Fitness

a = MLP['rFitness2_mlp']
b = MLI['rFitness2_mli']
c = THP['rFitness2_thp']
d = THI['rFitness2_thi']

sns.set()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (24, 6))
fig.suptitle('Distribution of betas from GWAS to fitness') 
sns.distplot(a, ax=ax1)
sns.distplot(b, ax=ax2)
sns.distplot(c, ax=ax3)
sns.distplot(d, ax=ax4)
ax1.set_xlabel('MLP')
ax2.set_xlabel('MLI')
ax3.set_xlabel('THP')
ax4.set_xlabel('THI')
fig.show()
#fig.savefig('Output/05_BetasDist.png', bbox_inches='tight', dpi=600) #same as Output/04_BetasDist.png

In [ ]:
# Sort & select
MLP = MLP.sort_values(by=['rFitness2_mlp'], ascending=False)
MLI = MLI.sort_values(by=['rFitness2_mli'], ascending=False)
THP = THP.sort_values(by=['rFitness2_thp'], ascending=False)
THI = THI.sort_values(by=['rFitness2_thi'], ascending=False)

# get the first and last 1000 objects (highest and lowest betas)
x=1000

selMLP = MLP.iloc[:x, :]   
selMLP = selMLP.append(MLP.iloc[-x:, :])
selMLPSNPs = selMLP['rs'].tolist()

selMLI = MLI.iloc[:x, :]   
selMLI = selMLI.append(MLI.iloc[-x:, :])
selMLISNPs = selMLI['rs'].tolist()

selTHP = THP.iloc[:x, :]   
selTHP = selTHP.append(THP.iloc[-x:, :])
selTHPSNPs = selTHP['rs'].tolist()

selTHI = THI.iloc[:x, :]   
selTHI = selTHI.append(THI.iloc[-x:, :])
selTHISNPs = selTHI['rs'].tolist()

In [ ]:
selTHI

In [ ]:
# check distribution of selection

a = selMLP['rFitness2_mlp']
b = selMLI['rFitness2_mli']
c = selTHP['rFitness2_thp']
d = selTHI['rFitness2_thi']

sns.set()
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (24, 6))
fig.suptitle('Distribution of selected betas from GWAS to fitness') 
sns.distplot(a, ax=ax1)
sns.distplot(b, ax=ax2)
sns.distplot(c, ax=ax3)
sns.distplot(d, ax=ax4)
ax1.set_xlabel('MLP')
ax2.set_xlabel('MLI')
ax3.set_xlabel('THP')
ax4.set_xlabel('THI')
fig.show()
#fig.savefig('Output/05_SelBetasDist.png', bbox_inches='tight', dpi=600)

In [ ]:
# Use all rs from the selection above and create new list
mySNPs = selMLPSNPs + selMLISNPs + selTHPSNPs + selTHISNPs

# check for duplicates
from collections import Counter
[k for k,v in Counter(mySNPs).items() if v>1]; 

#no output if cell ends with semicolon

In [ ]:
# remove duplicates
mySNPs = list(set(mySNPs))

# check again for duplicates
from collections import Counter
[k for k,v in Counter(mySNPs).items() if v>1]

In [ ]:
# create now target dataframe with selected SNPs

target = pd.DataFrame(mySNPs, columns=['rs'])

a = target.join(MLP.set_index('rs'), on='rs')
a.rename(columns={'rFitness2_mlp':'rFitness'}, inplace=True)
a['locat'] = 'MLP'

b = target.join(MLI.set_index('rs'), on='rs')
b.rename(columns={'rFitness2_mli':'rFitness'}, inplace=True)
b['locat'] = 'MLI'

c = target.join(THP.set_index('rs'), on='rs')
c.rename(columns={'rFitness2_thp':'rFitness'}, inplace=True)
c['locat'] = 'THP'

d = target.join(THI.set_index('rs'), on='rs')
d.rename(columns={'rFitness2_thi':'rFitness'}, inplace=True)
d['locat'] = 'THI'

target = a.append([b, c, d], ignore_index=True, sort=False)
target = target.reset_index()
target

In [ ]:
tarMLP = target[target["locat"] == 'MLP']
tarMLP #index 0-7756

In [ ]:
tarMLI = target[target["locat"] == 'MLI']
tarMLI #index 7757-15513

In [ ]:
tarTHP = target[target["locat"] == 'THP']
tarTHP #index 15514-23270

In [ ]:
tarTHI = target[target["locat"] == 'THI']
tarTHI #index 23271-31027

In [ ]:
# check distribution of target

sns.distplot(target['rFitness'])
plt.xlabel('Combined beta values as target variable')
plt.title('Distribution of target')
#plt.savefig('Output/05_TargetVarDist.png', bbox_inches='tight', dpi=600)

In [ ]:
predictors = pd.DataFrame(mySNPs, columns=['rs'])
predictors = predictors.join(betas.set_index('rs'), on='rs')
cols=[1,2,3,4]    #drop rFitness columns
predictors = predictors.drop(predictors.columns[cols], axis=1)
predictors = pd.concat([predictors]*4, ignore_index=True)
predictors

In [ ]:
# add annotation to predictors dataset

annot = pd.read_csv(f'/Carnegie/DPB/Data/Shared/Labs/Moi/Everyone/deepselection/naturedata/515g.ann.txt', sep='\t', names=['chr', 'ps', 'allel1', 'allel2', 'ann'])
annot['rs'] = annot.agg('{0[chr]}_{0[ps]}'.format, axis=1)    # create rs column out of chr and ps

predictors = predictors.join(annot.set_index('rs'), on='rs')
predictors = predictors.drop(columns=['chr', 'ps', 'allel1', 'allel2'])
predictors

In [ ]:
# encode annotation numerically
lb = LabelEncoder()
predictors['ann'] = lb.fit_transform(predictors['ann'])

# print encoding
lbMapping = dict(zip(lb.classes_, lb.transform(lb.classes_)))
#lbMapping

In [ ]:
# prepare climate data 

clim = pd.read_csv(f'/Carnegie/DPB/Data/Shared/Labs/Moi/Everyone/natvar/climate/2029gclimate.tsv', delim_whitespace=True)
climT = pd.concat([clim.iloc[[1813]]]*int(len(predictors)/2), ignore_index=True) #1813 = accession close to Tübingen
climM = pd.concat([clim.iloc[[1845]]]*int(len(predictors)/2), ignore_index=True) #1845 = accession close to Madrid
climFin = pd.concat([climM, climT], axis=0) #concat this way, to have Madrid at first, then Thübingen
climFin = climFin.iloc[:, :-12]    
climFin


In [ ]:
# finalize predictors dataset
predictors = pd.concat([predictors.reset_index(drop=True), climFin.reset_index(drop=True)], axis=1, sort=False)  # without reset_index, NAs were introduced in DF
predictors

In [ ]:
# check distribution of predictors

a = predictors['ann']
b = predictors['bio1']
c = predictors['clim-bio1']

sns.set()
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (24, 6))
fig.suptitle('Distribution of beta values from selected predictor variables') 
sns.distplot(a, ax=ax1)
sns.distplot(b, ax=ax2)
sns.distplot(c, ax=ax3)
ax1.set_xlabel('Annotation')
ax2.set_xlabel('Worldclim | bio1')
ax3.set_xlabel('Betas | bio1')
fig.show()
#fig.savefig('Output/05_PredVarDist.png', bbox_inches='tight', dpi=600)


## Random Forest
### Input variable preparation and distribution plots

In [ ]:
y = target['rFitness']
X = predictors.iloc[:, 1:].copy()    # without rs column

In [ ]:
X

### Packages

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn import metrics
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from yellowbrick.regressor import PredictionError, ResidualsPlot
from yellowbrick.features import Rank1D

***

In [ ]:
# Fit regression model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regr_rf = RandomForestRegressor(oob_score=True, random_state=0, n_estimators = 500)

In [ ]:
y_test2plot = y_test.copy()
y_test2plot = y_test2plot.reset_index()

#tarMLP #index 0-7756
#tarMLI #index 7757-15513
#tarTHP #index 15514-23270
#tarTHI #index 23271-31027

y_test2plot['locat'] = ['MLP' if 0 <= x <= 7756 else 'MLI' if 7757 <= x <= 15513 else 'THP' if 15514 <= x <= 23270 else 'THI' for x in y_test2plot['index']]
y_test2plot

In [ ]:
# Training
regr_rf.fit(X_train, y_train)

In [ ]:
# Prediction
y_rf = regr_rf.predict(X_test)

In [ ]:
predicted_train = regr_rf.predict(X_train)
predicted_test = regr_rf.predict(X_test)
test_score = r2_score(y_test, predicted_test)
spearman = spearmanr(y_test, predicted_test)
pearson = pearsonr(y_test, predicted_test)

In [ ]:
# Metrics
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_rf), file=open('Output/05_Metrics.txt', 'a'))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_rf), file=open('Output/05_Metrics.txt', 'a'))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_rf)), file=open('Output/05_Metrics.txt', 'a'))

print('Further statistics:', file=open('Output/05_Metrics.txt', 'a'))
print(f'Out-of-bag R2 score estimate: {regr_rf.oob_score_:>5.3}', file=open('Output/05_Metrics.txt', 'a'))
print(f'Test data R2 score: {test_score:>5.3}', file=open('Output/05_Metrics.txt', 'a'))
print(f'Test data Spearman correlation: {spearman[0]:.3}', file=open('Output/05_Metrics.txt', 'a'))
print(f'Test data Pearson correlation: {pearson[0]:.3}', file=open('Output/05_Metrics.txt', 'a'))

#### Plot results with Yellowbrick
https://www.scikit-yb.org/en/latest/api/regressor/peplot.html

In [ ]:
# Residuals plot
f = plt.figure()
visualizer = ResidualsPlot(regr_rf)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()
#f.savefig("Output/05_Residuals.png", bbox_inches='tight', dpi=600)

In [ ]:
# Prediction error plot
f = plt.figure()
visualizer = PredictionError(regr_rf)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()
#f.savefig("Output/05_PredActual.png", bbox_inches='tight', dpi=600)

In [ ]:
y_rf2plot = pd.DataFrame(y_rf)
df2plot = pd.concat([y_test2plot, y_rf2plot], axis=1)
df2plot.columns = ['index', 'actual', 'Location', 'pred']
df2plot
#df2plot.to_csv('Input/05_RF_2000selSNPs_predictedValues.csv')

In [ ]:
#df2plot = pd.read_csv('Input/05_RF_2000selSNPs_predictedValues.csv')

# order THP - THI - MLI - MLP
col = ['#ebac23','#b80058', '#006e00','#984ea3']


sns.set(rc={'figure.figsize':(13,10)})
sns.set_style("whitegrid")
sns.set_palette(col)
s = sns.scatterplot(x='pred', y='actual', hue='Location', sizes=(20), data=df2plot) 
plt.title("Random forest | 1,000 - 1,000 SNPs", size= 16, pad=25)
plt.suptitle("Actual vs predicted selection coefficients from MAD and TUE", size = 20)
plt.xlabel("Predicted", size=16)
plt.ylabel("Actual", size=16)
plt.xlim(-1,3)
plt.ylim(-1,3)
plt.setp(s.get_legend().get_texts(), fontsize='16') # for legend text
plt.setp(s.get_legend().get_title(), fontsize='18') # for legend title
#plt.savefig('Output/05_PredActual.png', bbox_inches='tight')




Feature importance:
https://machinelearningmastery.com/calculate-feature-importance-with-python/

In [ ]:
importance = regr_rf.feature_importances_         # get importance
labels = list(X.columns.values)

plt.figure(figsize=(18,8))
plt.title("Feature importance | 1,000 - 1,000 SNPs | MAD and TUE", size=20, pad=25)
plt.xlabel("Features", size=18)
plt.ylabel("Score", size=18)
imp = sns.barplot([x for x in range(len(importance))], importance, palette="viridis")
imp.set_xticklabels(labels,  rotation='vertical')
#plt.savefig('Output/05_Features.png', bbox_inches='tight')
plt.show()
